NARENDRA SINGH BISHT

AM.EN.P2ARI20043

M.Tech. AI (Semester I)

**Missionaries and Cannibals Problem**

Three missionaries and three cannibals are on one side of a river, along with a boat that can hold one or two people. Find a way to get everyone to the other side without ever leaving a group of missionaries in one place outnumbered by the cannibals in that place. This problem is famous in AI because it was the subject of the first paper that approached problem formulation from an analytical viewpoint (Amarel, 1968).

**2. Implement and solve the problem optimally using an appropriate search algorithm.**

Search Algorithm used: Depth-first Search

In [1]:
class Stack:
    
    #LIFO
    
    def __init__(self):
        self.stack = []
    
    def push(self, node):
        self.stack.append(node)
        
    def pop(self):
        if not self.empty():
            return self.stack.pop()
        else:
            raise Exception("Stack Empty!")
            
    def empty(self):
        return len(self.stack) == 0
    
    def contains_state(self, state):
        return any(node.state == state for node in self.stack)
    
    def __str__(self):
        result = f"Number of items in stack = {len(self.stack)}\n"
        for item in self.stack:
            result += f"{item}\n"
        return result 

In [2]:
class Node:
    def __init__(self, state, parent, action, depth):
        self.state = state
        self.parent = parent
        self.action = action
        self.depth = depth
    
    def repeated_state(self):
        if self.parent == None or self.parent.parent == None: 
            return False
        if self.parent.parent.state.equals(self.state): 
            return True
        return False
    
    def __str__(self):
        result = f"{self.state}"
        result += f" {self.depth}"
        if self.parent != None:
            result += f" {self.parent.state}"
            result += f" {self.action}"
        return result  

In [3]:
class Search:
    
    # Depth-first Search
    
    def __init__(self, start_state, goal_state):

        self.start_state = start_state
        self.goal_state = goal_state
            
    def find_solution(self):
        
        # Initialize fringe
        start_node = Node(self.start_state, None, None, 0)
        fringe = Stack()
        fringe.push(start_node)
        
        # Initialize an empty explored set
        self.explored = set()
        
        while not fringe.empty():
            
            # Choose a node from the fringe
            current_node = fringe.pop()
            
            # If the chosen node is the goal, then we have a solution
            if self.goal_state.equals(current_node.state):
                return current_node

            # Mark node as explored
            self.explored.add(current_node.state)
            
            # Add possible successors to the fringe
            successors = current_node.state.find_successors()
            for state, action in successors:   
                if not fringe.contains_state(state) and state not in self.explored and not state.illegal():
                    n = Node(state,
                             current_node,
                             action,
                             current_node.depth+1)
                    if n.repeated_state():
                        del(n)
                    else:
                        fringe.push(n)
            #print(fringe)
        return None
    
    def build_path(self, node):
        result = []
        while node:
            result.insert(0, node)
            node = node.parent
        return result
    
    def show_path(self, node):
        path = self.build_path(node)        
        for current_node in path:
            if current_node.action:
                print(current_node.action)
            print(current_node.state)
        print(f"Number of steps = {current_node.depth}")
    
    def solve(self):
        solution = self.find_solution()
        if solution == None:
            print("Search failed")
        else:
            print("Search completed successfully!")
            self.show_path(solution)

In [4]:
class MCB_State:
    
    """
    (#m, #c, #b)
    """

    def __init__(self, m_count, c_count, b_count):
        self.m_count = m_count
        self.c_count = c_count
        self.b_count = b_count
       
    def __str__(self):    
        return f"({self.m_count}, {self.c_count}, {self.b_count})"
    
    def equals(self, state):
        return (self.m_count == state.m_count and 
                self.c_count == state.c_count and 
                self.b_count == state.b_count)
    
    def illegal(self):
        if (self.m_count >= 0 
        and (3 - self.m_count) >= 0
        and self.c_count >= 0
        and (3 - self.c_count) >= 0):
            if self.m_count in [0, 3] or self.m_count == self.c_count:
                return False
            if self.m_count > self.c_count and (3 - self.m_count) > (3 - self.c_count):
                return False
        return True
    
    # There are five possible actions:
    # 1. Boat takes 1 missionary across river
    def take_1m(self):
        if self.b_count == 1:
            return MCB_State(self.m_count - 1, self.c_count, 0)
        return MCB_State(self.m_count + 1, self.c_count, 1)
    
    # 2. Boat takes 1 cannibal across the river
    def take_1c(self):
        if self.b_count == 1:
            return MCB_State(self.m_count, self.c_count - 1, 0)
        return MCB_State(self.m_count, self.c_count + 1, 1)
    
    # 3. Boat takes 2 missionaries across the river
    def take_2m(self):
        if self.b_count == 1:
            return MCB_State(self.m_count - 2, self.c_count, 0)
        return MCB_State(self.m_count + 2, self.c_count, 1)
    
    # 4. Boat takes 2 cannibals across the river
    def take_2c(self):
        if self.b_count == 1:
            return MCB_State(self.m_count, self.c_count - 2, 0)
        return MCB_State(self.m_count, self.c_count + 2, 1)
    
    # 5. Boat takes 1 missionary and 1 cannibal across the river
    def take_1m_1c(self):
        if self.b_count == 1:
            return MCB_State(self.m_count - 1, self.c_count - 1, 0)
        return MCB_State(self.m_count + 1, self.c_count + 1, 1)
        
    def find_successors(self):
        return [(self.take_1m(), "Boat takes 1 missionary across the river"), 
                (self.take_1c(), "Boat takes 1 cannibal across the river"),
                (self.take_2m(), "Boat takes 2 missionaries across the river"), 
                (self.take_2c(), "Boat takes 2 cannibals across the river"),
                (self.take_1m_1c(), "Boat takes 1 missionary and 1 cannibal across the river")]
    
s = Search(MCB_State(3,3,1), MCB_State(0,0,0))   
s.solve()

Search completed successfully!
(3, 3, 1)
Boat takes 1 missionary and 1 cannibal across the river
(2, 2, 0)
Boat takes 1 missionary across the river
(3, 2, 1)
Boat takes 2 cannibals across the river
(3, 0, 0)
Boat takes 1 cannibal across the river
(3, 1, 1)
Boat takes 2 missionaries across the river
(1, 1, 0)
Boat takes 1 missionary and 1 cannibal across the river
(2, 2, 1)
Boat takes 2 missionaries across the river
(0, 2, 0)
Boat takes 1 cannibal across the river
(0, 3, 1)
Boat takes 2 cannibals across the river
(0, 1, 0)
Boat takes 1 cannibal across the river
(0, 2, 1)
Boat takes 2 cannibals across the river
(0, 0, 0)
Number of steps = 11
